In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization,Activation, MaxPool2D, Concatenate
from tensorflow.keras.optimizers import Adam
from PIL import Image 
import numpy as np
import re
import os 
import glob 
import random
import logging
from tensorflow.keras import backend as K
import math
tf.get_logger().setLevel(logging.ERROR)

# Load model

In [2]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same", use_bias=False)(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


def build_unet(input_shape, n_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    c1, p1 = encoder_block(inputs, 16)
    c2, p2 = encoder_block(p1, 32)
    c3, p3 = encoder_block(p2, 64)
    c4, p4 = encoder_block(p3, 128)

    c5 = conv_block(p4, 256) #Bridge
    c6 = decoder_block(c5, c4, 128)
    c7 = decoder_block(c6, c3, 64)
    c8 = decoder_block(c7, c2, 32)
    c9 = decoder_block(c8, c1, 16)

    if n_classes == 1:  #Binary
        activation = 'sigmoid'
    else:
        activation = 'softmax'
    
    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(c9)
    model = Model(inputs, outputs, name="U-Net")
    return model

In [3]:
checkpoint_path = "/app/seg/models/v5_finetune.h5"
model = build_unet(input_shape=(128,128,3), n_classes=1)
model.load_weights(checkpoint_path)

# Representative dataset

In [4]:
def representative_dataset():
    dataset_dir = "/app/seg/data/images/train"
    files = glob.glob(dataset_dir + "/*.jpg")
    for file in files:
        img = tf.io.read_file(file)
        img = tf.image.decode_image(img, channels=3, dtype=tf.float32)
        resize_img = tf.image.resize_with_pad(img, target_height=128, target_width=128, method='bilinear')
        input_img = tf.expand_dims(resize_img, axis=0)
        yield [input_img]

In [5]:
model.input.set_shape((1,) + model.input.shape[1:])
# model.summary()

# Export tflite

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

save_path = "/app/seg/models/v5_finetune.tflite"

with open(save_path, 'wb') as f:
    f.write(tflite_model)

# Edgetpu compile

In [7]:
!edgetpu_compiler models/v5_finetune.tflite

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 652 ms.

Input model: models/v5_finetune.tflite
Input size: 1.92MiB
Output model: v5_finetune_edgetpu.tflite
Output size: 1.98MiB
On-chip memory used for caching model parameters: 1.96MiB
On-chip memory remaining for caching model parameters: 4.73MiB
Off-chip memory used for streaming uncached model parameters: 3.75KiB
Number of Edge TPU subgraphs: 1
Total number of operations: 45
Operation log: v5_finetune_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 
